In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\lapto\\OneDrive\\Desktop\\Face-reg'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Parth-cilans/Face-reg.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Parth-cilans"
os.environ["MLFLOW_TRACKING_PASSWORD"]="d506a17a9205f9a26d1fb8a4d483823c05d8ee8e"

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from Facerecognition.constants import *
from Facerecognition.utils.common import read_yaml,create_directories,save_json
from tensorflow.keras.applications.mobilenet import preprocess_input

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_preprocessing/preprocessed_images",
            all_params=self.params,
            mlflow_uri ="https://dagshub.com/Parth-cilans/Face-reg.mlflow",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
from urllib.parse import urlparse
import mlflow
import mlflow.keras

c:\Users\lapto\OneDrive\Desktop\Face-reg\faceenv\Lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
c:\Users\lapto\OneDrive\Desktop\Face-reg\faceenv\Lib\site-packages\pydantic\_internal\_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            preprocessing_function=preprocess_input,
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)


    def log_into_mlflow(self):
            mlflow.set_registry_uri(self.config.mlflow_uri)
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            
            with mlflow.start_run():
                mlflow.log_params(self.config.all_params)
                mlflow.log_metrics(
                    {"loss": self.score[0], "accuracy": self.score[1]}
                )
                # Model registry does not work with file store
                if tracking_url_type_store != "file":

                    # Register the model
                    # There are other ways to use the Model Registry, which depends on the use case,
                    # please refer to the doc for more information:
                    # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
                else:
                    mlflow.keras.log_model(self.model, "model") 

In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e


[2023-10-21 16:10:10,881: INFO: common yaml file: config\config.yaml loaded successfully]
[2023-10-21 16:10:10,881: INFO: common yaml file: params.yaml loaded successfully]
[2023-10-21 16:10:10,890: INFO: common created directory at: artifacts]
Found 10 images belonging to 4 classes.
10/10 [==============================] - 2s 150ms/step - loss: 0.7808 - accuracy: 0.8000
[2023-10-21 16:10:13,174: INFO: common json file saved at: scores.json]
[2023-10-21 16:10:13,174: INFO: common json file saved at: scores.json]


2023/10/21 16:10:17 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\lapto\AppData\Local\Temp\tmp8srm4waq\model\data\model\assets
[2023-10-21 16:10:18,709: INFO: builder_impl Assets written to: C:\Users\lapto\AppData\Local\Temp\tmp8srm4waq\model\data\model\assets]


c:\Users\lapto\OneDrive\Desktop\Face-reg\faceenv\Lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2023/10/21 16:13:15 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.
